### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation.
   - `cosine_similarity` from `sklearn.metrics.pairwise` for calculating item similarity.
   - `numpy` for numerical operations.

2. **Load Data**:
   - Load user-event interaction data from the Excel file `Combined_Interactions.xlsx`.

3. **Create Item-User Matrix**:
   - Pivot the data to create an **item-user interaction matrix**, where rows represent event IDs, columns represent user names, and values are interaction scores. Missing values are filled with 0.

4. **Compute Item Similarity**:
   - Use **cosine similarity** to compute a similarity matrix between events (item-item similarity) based on how users have interacted with those events.

5. **Recommendation Function**:
   - The `recommend_events()` function recommends events for a given user based on event similarity:
     - Retrieve events the user has already interacted with.
     - For each interacted event, find similar events using the **item-item similarity matrix**.
     - Check if the similar event has not been interacted with by the user.
     - Accumulate scores for similar events and return the top `n` most similar events.

6. **Sorting and Output**:
   - Recommendations are sorted based on similarity scores, and the top `n` (default 10) recommendations are returned.

7. **Example Usage**:
   - The function is called for the user **Pratik Bhande** to generate event recommendations.
   - The top recommendations are printed along with the user's name.

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_excel('Combined_Interactions.xlsx')

# Filter out events with interaction score of -1
#data = data[data['Interaction Score'] != -1]

# Pivot table to create item-user interaction matrix
item_user_matrix = data.pivot_table(index='Event ID', columns='Name', values='Interaction Score', fill_value=0)

# Compute item-item similarity matrix using cosine similarity
item_similarity = cosine_similarity(item_user_matrix)

# Function to recommend events for a given user based on item similarity
def recommend_events(user, top_n=10):
    # Get the events already interacted with by the user
    interacted_events = data[data['Name'] == user]['Event ID'].tolist()
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}

    # Iterate over the interacted events and recommend similar events
    for event_id in interacted_events:
        event_index = item_user_matrix.index.get_loc(event_id)
        similar_events = np.argsort(item_similarity[event_index])[::-1]  # Sort events by similarity
        
        for similar_event in similar_events:
            similar_event_id = item_user_matrix.index[similar_event]
            if similar_event_id not in interacted_events:
                if similar_event_id not in event_recommendations:
                    event_recommendations[similar_event_id] = item_similarity[event_index][similar_event]
                else:
                    event_recommendations[similar_event_id] += item_similarity[event_index][similar_event]
    
    # Sort recommendations by similarity score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Example: Recommend events for user 'Shreyas'
user = 'Pratik Bhande'
recommended_events = recommend_events(user)
print("Recommended events for {}: {}".format(user, recommended_events))


Recommended events for Pratik Bhande: [('EVT0030', 7.004089349457065), ('EVT0085', 6.437880653718111), ('EVT0122', 5.524320129548883), ('EVT0147', 4.913007800373261), ('EVT0022', 4.305465039190288), ('EVT0131', 4.135989903771232), ('EVT0081', 4.135989903771232), ('EVT0188', 4.135989903771232), ('EVT0140', 3.998255417005438), ('EVT0041', 3.947359782393053)]


In [1]:
# change interaction score for event event similarity